[![Notebook Tutorial](../__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/)

<img src='../__docs/__all/notebook_rules.png' />

In [79]:
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
% matplotlib notebook
import numpy as np
from scipy.ndimage import convolve

from ipywidgets import interact, widgets

from NeuNorm.normalization import Normalization

# Define input data files 

In [5]:
list_files = glob.glob('/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_*.tif')
for _file in list_files:
    assert os.path.exists(_file)

# Loading the data 

In [9]:
o_norm = Normalization()
o_norm.load(file=list_files, notebook=True)
data = o_norm.data['sample']['data']

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=10), Label(value='Time remaining:', layout=Layout(width='10%')), Label(value=' >> calculating << ')))

# Preview of data 

In [26]:
def plot(index):
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.imshow(data[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Crop 

In [37]:
x0 = 0
x1 = 460
y0 = 232
y1 = 285

crop_data = [_data[y0:y1, x0:x1] for _data in data]

def plot(index):
    fig, ax = plt.subplots(figsize=(12,2))
    cax = ax.imshow(crop_data[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Trying to reverse signal

Will take the max intensity and reverse all the values

```
new_pixel = max_value - old_pixel
```

In [44]:
max_value = np.max(crop_data)
crop_data_reversed = max_value - crop_data

def plot(index):
    fig, ax = plt.subplots(figsize=(12,2))
    cax = ax.imshow(crop_data_reversed[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Basic histogram segmentation 

In [46]:
threshold = 5
segmented_data = []
for _data in crop_data_reversed:
    _index = _data <= threshold
    _data[_index] = np.NaN
    segmented_data.append(_data)
    
def plot(index):
    fig, ax = plt.subplots(figsize=(12,2))
    cax = ax.imshow(segmented_data[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

/Users/j35/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less_equal
  after removing the cwd from sys.path.


interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Convolve data 

## matrix 3x3 

In [56]:
mean_kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]]) / 8.0
convolved_data = [convolve(_data, mean_kernel, mode='constant') for _data in segmented_data]

def plot(index):
    fig, ax = plt.subplots(2,1, figsize=(12,2))
    cax0 = ax[0].imshow(convolved_data[index], cmap='viridis')
    
    cax1 = ax[1].imshow(segmented_data[index])
    
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

## maxtrix 5x5

In [66]:
mean_kernel = np.array([[1, 1, 1, 1, 1], 
                        [1, 2, 2, 2, 1], 
                        [1, 2, 0, 2, 1], 
                        [1, 2, 2, 2, 1], 
                        [1, 1, 1, 1, 1]]) / 32.0
convolved_data = [convolve(_data, mean_kernel, mode='constant') for _data in segmented_data]

def plot(index):
    fig, ax = plt.subplots(2,1, figsize=(12,2))
    cax0 = ax[0].imshow(convolved_data[index], cmap='viridis')
    
    cax1 = ax[1].imshow(segmented_data[index])
    
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Profile 

In [98]:
profile = []
for _data in convolved_data:
    _profile = np.nansum(_data, axis=0)
    profile.append(_profile)
    
def plot(index):
    fig4 = plt.figure(figsize=(12,5))
    spec4 = gridspec.GridSpec(ncols=2, nrows=3)
    
    ax = fig4.add_subplot(spec4[:, 0])
    cax1 = ax.plot(profile[index])

    ax = fig4.add_subplot(spec4[0, 1])
    cax1 = ax.imshow(segmented_data[index], cmap='viridis')

    ax = fig4.add_subplot(spec4[1, 1])
    cax1 = ax.imshow(convolved_data[index], cmap='viridis')
    
    
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))


interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>